In [10]:
#Import Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from sqlalchemy import create_engine
import pymysql
from scipy.misc import derivative

In [11]:
pymysql.install_as_MySQLdb()

In [12]:
#Creating dataframes
bitcoin = pd.read_csv('./crypto/bitcoin_price.csv')
ibm = pd.read_csv('./dow/IBM.csv')

ibm.head()

,date,open,high,low,close,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime
0,1/9/2014,158.5454,158.9480,156.4736,157.1698,4322284,4322284,-0.494880,-0.314,157.4492,9-Jan-14,0.000000
1,1/10/2014,157.9499,158.1680,156.2472,157.0692,4023089,4023089,-0.100653,-0.064,156.9879,10-Jan-14,-0.000640
2,1/13/2014,156.2304,156.8092,154.2173,154.4690,5789107,5789107,-2.600200,-1.655,154.8575,13-Jan-14,-0.017184
3,1/14/2014,155.2239,156.3713,154.2356,155.9452,4619509,4619509,1.476200,0.956,155.6201,14-Jan-14,-0.007792
4,1/15/2014,155.8613,158.2351,155.5845,157.4718,4829291,4829291,1.526600,0.979,157.5421,15-Jan-14,0.001921


In [13]:
rds_connection_string = "guest@localhost/etl_project"
engine = create_engine(f'mysql://{rds_connection_string}') 

In [14]:
#Creating Bitcoin and IBM tables under 'etl_project' database in MySQL
bitcoin.to_sql('bitcoin', con=engine, if_exists='replace')
ibm.to_sql('ibm', con=engine, if_exists='replace')

OperationalError: (pymysql.err.OperationalError) (1045, "Access denied for user 'guest'@'localhost' (using password: NO)") (Background on this error at: http://sqlalche.me/e/e3q8)

In [15]:
bitcoin_df = bitcoin[['Date','Close','Volume']]
bitcoin_df = bitcoin_df.rename(columns={'Close': 'Bitcoin Close', 'Volume':'Bitcoin Volume'})

ibm_df = ibm[['date','close','volume']]
ibm_df = ibm_df.rename(columns={'date': 'Date', 'close': 'IBM Close', 'volume': 'IBM Volume'})

comparison_df = pd.merge(ibm_df, bitcoin_df, on='Date')
comparison_df = comparison_df.set_index('Date')
comparison_df

,IBM Close,IBM Volume,Bitcoin Close,Bitcoin Volume
Date,,,,
1/9/2014,157.1698,4322284,846.86,"59,998,600"
1/10/2014,157.0692,4023089,868.48,"31,876,800"
1/13/2014,154.4690,5789107,841.20,"45,580,900"
1/14/2014,155.9452,4619509,833.27,"20,829,800"
1/15/2014,157.4718,4829291,860.90,"28,107,200"
1/16/2014,158.3273,4775167,835.63,"19,149,300"
1/17/2014,159.4429,7671146,814.64,"39,031,700"
1/21/2014,158.0505,11121410,863.91,"18,997,300"
1/22/2014,152.8669,13800586,845.59,"18,453,700"


In [16]:
comparison_df.to_sql('ibm vs bitcoin', con=engine, if_exists='replace')

OperationalError: (pymysql.err.OperationalError) (1045, "Access denied for user 'guest'@'localhost' (using password: NO)") (Background on this error at: http://sqlalche.me/e/e3q8)

NameError: name 'comparison_df' is not defined